In [1]:
import numpy as np 
import pandas as pd 

import os

In [2]:
from music21 import converter, corpus, instrument, midi, note, chord, pitch,stream,roman

In [3]:
def concat_path(path, child):
    return path + "/" + child

def open_midi(midi_path, remove_drums):
    # There is an one-line method to read MIDIs
    # but to remove the drums we need to manipulate some
    # low level MIDI events.
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()
    if (remove_drums):
        for i in range(len(mf.tracks)):
            mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          

    return midi.translate.midiFileToStream(mf)

base_midi = open_midi(concat_path('김광석', "거리에서_김광석.mid"), True)
base_midi

C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'\xb0\xc5\xb8\xae\xbf\xa1\xbc\xad'>; getting generic Instrument
  warnings.warn(


<music21.stream.Score 0x159e03eebb0>

In [4]:
def process_single_file(midi_param):
    try:
        game_name = midi_param[0]
        midi_path = midi_param[1]
        midi_name = get_file_name(midi_path)
        midi = open_midi(midi_path, True)
        return (
            midi.analyze('key'),
            game_name,
            harmonic_reduction(midi),
            midi_name)
    except Exception as e:
        print("Error on {0}".format(midi_name))
        print(e)
        return None

In [5]:
def note_count(measure, count_dict):
    bass_note = None
    for chord in measure.recurse().getElementsByClass('Chord'):
        # All notes have the same length of its chord parent.
        note_length = chord.quarterLength
        for note in chord.pitches:          
            # If note is "C5", note.name is "C". We use "C5"
            # style to be able to detect more precise inversions.
            note_name = str(note) 
            if (bass_note is None or bass_note.ps > note.ps):
                bass_note = note
                
            if note_name in count_dict:
                count_dict[note_name] += note_length
            else:
                count_dict[note_name] = note_length
        
    return bass_note
                
def simplify_roman_name(roman_numeral):
    # Chords can get nasty names as "bII#86#6#5",
    # in this method we try to simplify names, even if it ends in
    # a different chord to reduce the chord vocabulary and display
    # chord function clearer.
    ret = roman_numeral.romanNumeral
    inversion_name = None
    inversion = roman_numeral.inversion()
    
    # Checking valid inversions.
    if ((roman_numeral.isTriad() and inversion < 3) or
            (inversion < 4 and
                 (roman_numeral.seventh is not None or roman_numeral.isSeventh()))):
        inversion_name = roman_numeral.inversionName()
        
    if (inversion_name is not None):
        ret = ret + str(inversion_name)
        
    elif (roman_numeral.isDominantSeventh()): ret = ret + "M7"
    elif (roman_numeral.isDiminishedSeventh()): ret = ret + "o7"
    return ret
def harmonic_reduction(midi_file):
    ret = []
    temp_midi = stream.Score()
    temp_midi_chords = midi_file.chordify()
    temp_midi.insert(0, temp_midi_chords)    
    music_key = temp_midi.analyze('key')
    max_notes_per_chord = 4   
    for m in temp_midi_chords.measures(0, None): # None = get all measures.
        if (type(m) != stream.Measure):
            continue
        
        # Here we count all notes length in each measure,
        # get the most frequent ones and try to create a chord with them.
        count_dict = dict()
        bass_note = note_count(m, count_dict)
        if (len(count_dict) < 1):
            ret.append("-") # Empty measure
            continue
        
        sorted_items = sorted(count_dict.items(), key=lambda x:x[1])
        sorted_notes = [item[0] for item in sorted_items[-max_notes_per_chord:]]
        measure_chord = chord.Chord(sorted_notes)
        
        # Convert the chord to the functional roman representation
        # to make its information independent of the music key.
        roman_numeral = roman.romanNumeralFromChord(measure_chord, music_key)
        ret.append(simplify_roman_name(roman_numeral))
        
    return ret

In [6]:
key_signature_column = []
harmonic_reduction_column = []
midi_name_column = []

for file in os.listdir('schubert'):
    midi_name_column.append(file)
    
    midifile = open_midi(concat_path('schubert', file), True)
    try:
        key_signature_column.append(midifile.analyze('key'))
        harmonic_reduction_column.append(harmonic_reduction(midifile))
    except:
        print('no pitches')

C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2009 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2010 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2007 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music2

In [7]:
d = {'midi_name': midi_name_column,
     'composer' : 'schubert',
         'key_signature' : key_signature_column,
         'harmonic_reduction': harmonic_reduction_column}
    
df_schubert = pd.DataFrame(data=d)

In [8]:
df_schubert.head()

,midi_name,composer,key_signature,harmonic_reduction
0,schubert_D850_1.mid,schubert,D major,"[I53, I7, I, I, i53, I7, #iv43, bIII64, v, v, ..."
1,schubert_D850_2.mid,schubert,A major,"[i53, I53, IV, V65, IV64, I53, V, iii64, I53, ..."
2,schubert_D850_3.mid,schubert,D major,"[V7, I6, I53, V6, vii7, vi6, II7, V, bVI65, ii..."
3,schubert_D850_4.mid,schubert,D major,"[I64, I64, V7, iii, IV, I64, V7, iv65, iii, I,..."
4,schubert_D935_1.mid,schubert,f minor,"[i, I53, i, V65, ii65, V, i, I53, i, #v7, v42,..."


In [9]:
key_signature_column = []
harmonic_reduction_column = []
midi_name_column = []

for file in os.listdir('mozart'):
    midi_name_column.append(file)
    
    midifile = open_midi(concat_path('mozart', file), True)
    try:
        key_signature_column.append(midifile.analyze('key'))
        harmonic_reduction_column.append(harmonic_reduction(midifile))
    except:
        print('no pitches')

d = {'midi_name': midi_name_column,
     'composer' : 'mozart',
         'key_signature' : key_signature_column,
         'harmonic_reduction': harmonic_reduction_column}
    
df_mozart = pd.DataFrame(data=d)

C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2006 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 1997 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2005 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <

In [10]:
df_mozart.head(1)

,midi_name,composer,key_signature,harmonic_reduction
0,mz_311_1.mid,mozart,D major,"[ii42, I6, I, ii42, I6, I, iv43, ii, V7, vi, I..."


In [33]:
key_signature_column = []
harmonic_reduction_column = []
midi_name_column = []

for file in os.listdir('haydn'):
    midi_name_column.append(file)
    
    midifile = open_midi(concat_path('haydn', file), True)
    try:
        key_signature_column.append(midifile.analyze('key'))
        harmonic_reduction_column.append(harmonic_reduction(midifile))
    except:
        print('no pitches')

d = {'midi_name': midi_name_column,
     'composer' : 'haydn',
         'key_signature' : key_signature_column,
         'harmonic_reduction': harmonic_reduction_column}
    
df_haydn = pd.DataFrame(data=d)

C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2010 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2009 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 1998 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(


In [34]:
df_haydn.head(1)

,midi_name,composer,key_signature,harmonic_reduction
0,haydn_33_1.mid,haydn,D major,"[I64, I, vi65, I, I53, I65, IV53, iii65, I7, I..."


In [35]:
key_signature_column = []
harmonic_reduction_column = []
midi_name_column = []

for file in os.listdir('chopin'):
    midi_name_column.append(file)
    
    midifile = open_midi(concat_path('chopin', file), True)
    try:
        key_signature_column.append(midifile.analyze('key'))
        harmonic_reduction_column.append(harmonic_reduction(midifile))
    except:
        print('no pitches')

d = {'midi_name': midi_name_column,
     'composer' : 'chopin',
         'key_signature' : key_signature_column,
         'harmonic_reduction': harmonic_reduction_column}
    
df_chopin = pd.DataFrame(data=d)

C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2002 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 1997 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 1997 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music2

In [36]:
df_chopin.head(1)

,midi_name,composer,key_signature,harmonic_reduction
0,chpn-p1.mid,chopin,C major,"[vi42, ii, vi42, I42, vi, vi, iii7, vii7, vi42..."


In [37]:
key_signature_column = []
harmonic_reduction_column = []
midi_name_column = []

for file in os.listdir('beeth'):
    midi_name_column.append(file)
    
    midifile = open_midi(concat_path('beeth', file), True)
    try:
        key_signature_column.append(midifile.analyze('key'))
        harmonic_reduction_column.append(harmonic_reduction(midifile))
    except:
        print('no pitches')

d = {'midi_name': midi_name_column,
     'composer' : 'beeth',
         'key_signature' : key_signature_column,
         'harmonic_reduction': harmonic_reduction_column}
    
df_beeth = pd.DataFrame(data=d)

C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2001 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2008 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2009 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(
C:\Users\medici\Anaconda3\lib\site-packages\music21\midi\translate.py:883: TranslateWarning: Unable to determine instrument from <musi

In [38]:
df_beeth.head(1)

,midi_name,composer,key_signature,harmonic_reduction
0,appass_1.mid,beeth,f minor,"[v, #ii42, #ii42, #ii42, v, iv, iv, #i, i, #ii..."


In [39]:
df = pd.concat([df_schubert,df_mozart,df_haydn,df_chopin,df_beeth])

In [40]:
df

,midi_name,composer,key_signature,harmonic_reduction
0,schubert_D850_1.mid,schubert,D major,"[I53, I7, I, I, i53, I7, #iv43, bIII64, v, v, ..."
1,schubert_D850_2.mid,schubert,A major,"[i53, I53, IV, V65, IV64, I53, V, iii64, I53, ..."
2,schubert_D850_3.mid,schubert,D major,"[V7, I6, I53, V6, vii7, vi6, II7, V, bVI65, ii..."
3,schubert_D850_4.mid,schubert,D major,"[I64, I64, V7, iii, IV, I64, V7, iv65, iii, I,..."
4,schubert_D935_1.mid,schubert,f minor,"[i, I53, i, V65, ii65, V, i, I53, i, #v7, v42,..."
...,...,...,...,...
24,pathetique_2.mid,beeth,A- major,"[iii7, iii64, #vii42, V53, vii, VI7, #iii42, v..."
25,pathetique_3.mid,beeth,c minor,"[i, i, i, i53, V6, #v6, i53, III65, iv, #v42, ..."
26,waldstein_1.mid,beeth,a minor,"[III53, III53, VII6, VII6, bII53, bII53, VI6, ..."
27,waldstein_2.mid,beeth,C major,"[IV7, III, #i65, biii, ii42, I7, ii53, I7, IV5..."


In [41]:
df.to_csv('작곡가데이터프레임.csv', index=False, encoding='cp949')